# CSC337: Coursework 1

## Task 1: Design 4
### Measuring national power generation's effect on the country's CO₂ emissions
<b>Visual Design Type:</b> Log-based scatter plot with line of regression

<b>Name of Tool:</b> Altair

<b>Country:</b> Worldwide

<b>Year:</b> 2014

<b>Visual Mappings:</b>
+ ?

Additionally, the following points were deliberated:
+ ?

In [1]:
import pandas as pd
import numpy as np
import altair as alt

Debug options

In [2]:
# font to use for chart labels
__CHART_FONT__ = 'Circular'

# DEBUG: disable maximum row prevention (cripples chart performance)
# alt.data_transformers.disable_max_rows()

# DEBUG: set max rows/columns in pandas table previewer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

Load the GPPD data set and sequentially derive the required information

In [3]:
# load GPPD data set into pandas array and keep only required columns
d_all_plants = pd.read_csv('../data/global_power_plant_database.csv')[
    ['country', 'country_long', 'primary_fuel', 'estimated_generation_gwh', 'generation_gwh_2014']
]

# combine national gen figures with US EPA figures for more complete figures
d_all_plants['plant_gen'] = d_all_plants['estimated_generation_gwh'].fillna(
    d_all_plants['generation_gwh_2014']
)

# remove uncombined columns
d_all_plants.drop(
    columns=['estimated_generation_gwh', 'generation_gwh_2014'],
    inplace=True
)

# remove null data from combined column
d_all_plants.dropna(
    subset=['plant_gen'],
    inplace=True
)

# find total power gen for each country
d_plants = d_all_plants.groupby(['country', 'country_long']).sum().reset_index()

# find fuel type with most production per country and add it to plants table
d_plants_most = d_all_plants.groupby(['country', 'primary_fuel'])['plant_gen'].sum()

d_plants = d_plants.merge(
    d_plants_most.loc[d_plants_most.groupby(level=0).idxmax()].reset_index().drop(
        columns=['plant_gen']
    ),
    how='inner',
    on='country'
)

In [4]:
# load ISO country code translation table
d_iso_codes = pd.read_excel('../data/ISO_3166_lookup.xlsx')

# append ISO numeric country codes to table
d_plants = d_plants.merge(
    d_iso_codes[['Alpha-3', 'Numeric']],
    left_on='country',
    right_on='Alpha-3'
).drop(
    columns='Alpha-3'
)

In [5]:
# load UN global emissions data
d_emissions = pd.read_csv(
    '../data/SYB60_312_Carbon Dioxide Emission Estimates.csv',
    header=1
).drop(
    columns=['Unnamed: 1', 'Footnotes', 'Source']
)

# select only 2014 figures
d_emissions = d_emissions[d_emissions['Year'] == 2014]

# select only national totals then remove unnecessary columns
d_emissions = d_emissions[~d_emissions['Series'].str.contains('per capita')].drop(
    columns=['Year', 'Series']
).reset_index(
    drop=True
)

# give more concise column names
d_emissions.columns = ['region', 'co2_kt']

# merge emissions data into main table
d_plants = d_plants.merge(
    d_emissions.reset_index(drop=True),
    how='left',
    left_on='Numeric',
    right_on='region'
).drop(
    columns=['country', 'Numeric', 'region']
)

In [6]:
d_plants['co2_kt'] = d_plants['co2_kt'].str.replace(',', '')

d_plants['co2_kt'] = d_plants['co2_kt'].astype(float)

# truncate all figures for nicer presentation
d_plants = d_plants.round(0)

Draw the visualisation

In [7]:
# create the points on the graph, indicating countries
marks = alt.Chart(
    data=d_plants
).mark_circle(
    size=100
).encode(
    x=alt.X(
        'plant_gen:Q',
        axis=alt.Axis(
            title="National electricity generation (GWh)",
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=14,
            labelFont=__CHART_FONT__,
            labelFlush=False
        ),
        scale=alt.Scale(type='log')
    ),
    y=alt.Y(
        'co2_kt:Q',
        axis=alt.Axis(
            title='National CO₂ emissions (kTonne)',
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=14,
            labelFont=__CHART_FONT__,
        ),
        scale=alt.Scale(
            type='log'
        )
    ),
    color=alt.Color(
        'primary_fuel:N',
        scale=alt.Scale(
            range=['#3F51B5', '#8BC34A', '#9C27B0', '#FF9800', '#E91E63', '#795548', '#03A9F4']
        ),
        legend=alt.Legend(
            title='Main fuel type',
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=16,
            labelFont=__CHART_FONT__
        )
    ),
    tooltip=[
        alt.Tooltip(
            field='country_long',
            title='Country',
            type='nominal'
        ),
        alt.Tooltip(
            field='primary_fuel',
            title='Main fuel type',
            type='nominal'
        ),
        alt.Tooltip(
            field='co2_kt',
            title='CO₂ emissions (kT)',
            type='quantitative'
        ),
        alt.Tooltip(
            field='plant_gen',
            title='National electricity generation (GWh)',
            type='quantitative'
        )
    ]
)

# add a line of regression: y = a * xb
line = alt.Chart(
    d_plants
).transform_regression(
    'plant_gen', 'co2_kt',
    method='pow'
).mark_line(
    color="#607D8B"
).encode(
    x='plant_gen',
    y='co2_kt'
)

alt.layer(
    marks,
    line,
    height=750,
    width=1000,
    padding=20,
    title=alt.TitleParams(
        text='Effects of Electricity Generation on National CO₂ Emissions',
        fontSize=22,
        font=__CHART_FONT__
    )
)

alt.LayerChart(...)